In [1]:
import os
import numpy as np
import pandas as pd
import pickle5 as pkl
import plotly.express as px
import plotly.graph_objects as go

from keypoint_util import KeypointSelector

folder = os.path.join('ndi_servo', '2022-04-13-morning', '260650')
file1 = '260650-2022-04-13-14-40-08-Servo-displacement_postprocessed'
file2 = '260650-2022-04-13-14-43-37_postprocessed.csv'
# with open(os.path.join(data_folder, data497, 'labeled_actions_new.pkl'), 'rb') as f:
#     label_actions = pkl.load(f)
# trajectory_file = os.path.join(data_folder, data497, 'obj_pose_trajectory_interpolated_median.h5')
# trajectory_dataset = pd.read_hdf(trajectory_file)
all_objs = ['tap', 'pitcher', 'cup', 'teabag1', 'teabag2']
grip_file = os.path.join(folder, file1)
state_file = os.path.join(folder, file2)
grip_states = pd.read_csv(grip_file)
pos_states = pd.read_csv(state_file, index_col='Unnamed: 0')

In [2]:
def attach_grip_states(df, grip):
    new_df = df.copy()
    new_df['action'] = None
    for i, row in grip.iterrows():
        check_time = row['Timestamp']
        new_df.iloc[(df['Time'] - check_time).abs().argsort()[:1], -1] = row['Gripper state']
    return new_df

dframe = attach_grip_states(pos_states, grip_states)

In [19]:
selector = KeypointSelector(velocity_threshold=20, rotation_threshold=0.05, rotation_max=.7, rotation_window_size=10)
cond_df, keypoints, velo_df, rot_df = selector.search(dframe)

In [20]:
# List all close and open grasp frame indices.
grasp_indices = cond_df.index[cond_df['condition'] == 'grasp'].tolist()
durations = [grasp_indices[i*2:i*2+2] for i in range(int(len(grasp_indices)/2))]
print(durations)

[[47, 107], [192, 313], [373, 482], [542, 722], [782, 862], [1058, 1441]]


In [21]:
# 47 - 107: teabag to cup
# 192- 313: pitcher to tap
# 373- 482: turn tap
# 542- 722: wait
# 782- 862: turn tap back
# 1058- 1441: pour pitcher to cup

In [22]:
# Set start and end frames
start, end = durations[3]

traj_df = cond_df.copy().iloc[start:end+1]
keypoint_df = cond_df.copy().iloc[keypoints]
keypoint_sliced_df = keypoint_df.loc[[i for i in keypoint_df.index if i in range(start, end+1)]]
fig1 = px.line_3d(traj_df, x='x', y='y', z='z')
fig2 = px.scatter_3d(keypoint_sliced_df, x='x', y='y', z='z', color='Time', symbol='condition')
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.update_traces(marker=dict(size=5))
fig3.show()

In [17]:
keypoint_sliced_df

,Time,x,y,z,Rx,Ry,Rz,action,condition
1058,80.913628,75.874687,42.994158,-1621.303447,1.755916,0.989506,1.510982,closed,grasp
1070,81.913685,53.224350,38.365842,-1582.274467,1.731183,1.039954,1.545579,None,rotation
1073,82.163700,24.504253,35.291911,-1525.596966,1.658489,1.126093,1.550173,None,rotation
1074,82.249705,18.683425,33.836679,-1511.034842,1.640236,1.148090,1.545523,None,velocity
1087,83.330767,-40.020993,-100.231229,-1224.196314,1.305067,1.488078,1.391753,None,rotation
1088,83.414771,-28.745060,-114.821040,-1210.136396,1.318085,1.478871,1.386180,None,velocity
1089,83.498776,-22.607887,-121.890491,-1204.621454,1.325326,1.469153,1.384285,None,rotation
1349,105.899057,-245.822549,-227.933011,-1376.011446,-0.005853,0.539395,1.429571,None,rotation
1352,106.250077,-245.616562,-227.551430,-1375.090193,0.002964,0.543209,1.425984,None,rotation
1360,107.083125,-171.438266,-196.312243,-1287.297432,0.527538,0.807510,1.492644,None,velocity


In [8]:
target = 264
print('Smoothed velocity:\n', velo_df.iloc[target-1:target+2])
print('Smoothed rotation:\n', rot_df.iloc[target-1:target+2])
print(cond_df.iloc[target-1:target+2])

Smoothed velocity:
 263    35.301597
264    32.132049
265    27.604682
dtype: float64
Smoothed rotation:
            Rx        Ry        Rz
263  0.022587  0.017705  0.017431
264  0.018871  0.016338  0.017424
265  0.014918  0.013854  0.016955
          Time          x          y            z        Rx        Ry  \
263  17.694012   0.505626 -88.768841 -1438.456863  1.714720  1.629062   
264  17.776017  11.110939 -83.269223 -1469.829157  1.735225  1.613045   
265  17.860022  25.495676 -76.627143 -1498.278738  1.761039  1.611274   

           Rz action condition  
263  1.621598   None      None  
264  1.599658   None      None  
265  1.566727   None      None  
